<a href="https://colab.research.google.com/github/yuvalofek/FrequentistML/blob/master/FreqML3_Model_Assesment%26Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Model Assesment and Selection**

Yuval Epstain Ofek


---




**The Prompt:**
Re-implement the example in section 7.10.2 using any simple, out of the box classifier (like K nearest neighbors from sci-kit). Reproduce the results for the incorrect and correct way of doing cross-validation.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as nearNC
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#parameters
N = 50
p = 5000
K = 5
Highest_N = 100

#create dataset 
np.random.seed(1)
y = 1*(np.random.rand(1,N)>0.5)
x = np.random.randn(p,N)

Num_samp = int(N/K)
y_rav = np.ravel(y) #needed to call the k-nearest neighbors classifier

#checking sizes
print(y.shape, x.shape)

(1, 50) (5000, 50)


**The Wrong Way...**

In [ ]:
#cross correlation:
corr_xy = np.ravel(x@y.T)
#Use indices of the x values with the highest correlation
mx_i = corr_xy.argsort()[-Highest_N:][::-1]
X_new = x[mx_i,:].T

correct = []
for i in range(K):
  #split up the X input into training and validation sets, based on the current k
  val = X_new[i*Num_samp:(i+1)*Num_samp,:]
  y_val = y_rav[i*Num_samp:(i+1)*Num_samp]
  tr = np.concatenate([X_new[:i*Num_samp,:],X_new[(i+1)*Num_samp:,:]])
  y_tr = np.concatenate([y_rav[:i*Num_samp],y_rav[(i+1)*Num_samp:]])
  #Train the 1-nearest neighbors classifier
  neigh = nearNC(n_neighbors=1)
  neigh.fit(tr,np.ravel(y_tr))
  # check how many of the validation set we got right
  correct.append(neigh.predict(val) == y_val)
print("percent error: " + str(round((1-np.mean(correct))*100)))

percent error: 14.0


**The Correct Way**

In [ ]:
correct_ = []
for i in range(K):
  #spllit up the X input into training and validation sets, based on the current k
  val = x.T[i*Num_samp:(i+1)*Num_samp,:]
  y_val = y_rav[i*Num_samp:(i+1)*Num_samp]
  tr = np.concatenate([x.T[:i*Num_samp,:],x.T[(i+1)*Num_samp:,:]])
  y_tr = np.concatenate([y_rav[:i*Num_samp],y_rav[(i+1)*Num_samp:]])

  #correlation coefficients:
  corr_xy  = np.ravel(tr.T@y_tr.reshape((1,-1)).T)

  #get indices of the highest correlated predictors
  mx_i = corr_xy.argsort()[-Highest_N:][::-1]
  tr_mx = tr[:,mx_i]
  val_mx = val[:,mx_i]
  

  #Train the 1-nearest neighbors classifier
  neigh = nearNC(n_neighbors=1)
  neigh.fit(tr_mx,np.ravel(y_tr))
  # check how many of the validation set we got right
  correct_.append(neigh.predict(val_mx) == y_val)
print('percent error: ' + str((1-np.mean(correct_))*100))

percent error: 46.0


We see results similar to the example in section 7.10.2, the error for the incorrect way is extremely small, more so than it should be. Once we correct our mistakes, we reach a value much closer to 0.5, the predicted value. 